In [22]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
import pandas as pd
import os

from itertools import cycle

def return_response(url):
    headers = {
            'sec-fetch-mode': 'cors',
            'accept-language': 'en-US,en;q=0.9',
            'authorization': '',
            'x-requested-with': 'XMLHttpRequest',
            'appos': 'web',
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36',
            'accept': '*/*',
            'authority': 'stockx.com',
            'sec-fetch-site': 'same-origin',
            'appversion': '0.1',
    'accept': 'application/json'}
    
    return requests.get(url, headers=headers)

def play_alert():
    file = './/sms-alert-3-daniel_simon.mp3'
    os.system("afplay " + file)

def return_page_has_sale_price(page_response):
    page_content = BeautifulSoup(page_response.content, "html.parser")
    soup_page_content = page_content.findAll("div", {"class": "last-sale-value"})
    
    if len(soup_page_content) != 0 and page_response.ok == True:
        print('page loaded succesfully')
        return True
    
    else:
        print('page did not load')
        return False
            
def scrape_retail_price(url):
    page_response = return_response(url)
    if return_page_has_sale_price(page_response) == True:
        soup = BeautifulSoup(page_response.content, "html.parser")
        data = soup.findAll("span", {"class": "sneak-score"})
        split_price = data[1].text
        retail_price = split_price.split('$')[-1]
        return retail_price
    
    else:
        play_alert()
        print(f'broke on url {url}')
        raise SystemExit

In [19]:
nike_df = pd.read_csv('persistence/nikes.clean.csv')

In [20]:
urls = nike_df['url']

In [21]:
scrape_retail_price(test_url)

page loaded succesfully
200


'200'

In [12]:
retail_prices = {}
wait_seconds = [1, 2, 4, 5, 6, 7]
wait_pool = cycle(wait_seconds)

In [13]:
for url in (urls):
    if url not in retail_prices:
        try:
            wait = next(wait_pool)
            time.sleep(wait)
            print(url)
            retail_prices[url] = scrape_retail_price(url)
        except requests.exceptions.MissingSchema:
            print('missing schema')
            pass

https://stockx.com/nike-sb-dunk-low-raygun-tie-dye
page loaded succesfully
100
https://stockx.com/nike-dunk-low-off-white-pine-green
page loaded succesfully
170


KeyboardInterrupt: 

In [14]:
len(retail_prices)

2

In [15]:
retail_prices

{'https://stockx.com/nike-sb-dunk-low-raygun-tie-dye': '100',
 'https://stockx.com/nike-dunk-low-off-white-pine-green': '170'}

In [43]:
price_df = pd.DataFrame(list(retail_prices.items()), columns=['url', 'retailPrice($)'])

In [44]:
df_merged = pd.merge(nike_df, price_df, on='url', how='outer')

In [45]:
df_merged.iloc[20]

color                        Black/Metallic Silver-White-Pink Blast
name                    Nike Zoom Terra Kiger 5 Off-White Black (W)
sku                            875fb3df-eb58-4192-bfe6-c89203aa80f7
itemCondition                       https://schema.org/NewCondition
releaseDate                                     2019-06-24 23:59:59
image             https://stockx.imgix.net/Nike-Zoom-Terra-Kiger...
brand                                                          Nike
model                                       Nike Zoom Terra Kiger 5
url               https://stockx.com/nike-zoom-terra-kiger-5-off...
retailPrice($)                                                  180
Name: 20, dtype: object

In [46]:
df_merged.to_csv('persistence/nikes.out.csv', index=False)